In [4]:
import os
import pandas as pd
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
root_dir = "data/refit"
datetime_temp = "%Y-%m-%d %H:%M:%S"
pd.set_option('display.max_rows', None)
def avg_duration_evt(data_x, data_y, onthreshold, min_on = timedelta(minutes=1)):
    pre = data_x[0]
    state = "off"
    on = []
    for i,d in enumerate(data_y):
        if i > 0 and data_x[i] - data_x[i-1] > timedelta(hours=1):
            if state == "on":
                state = "off"
                this_on = data_x[i] - pre
                if this_on < min_on:
                    continue
                on.append(this_on.seconds//60)
                if data_x[i-1] - pre > timedelta(hours=10):
                    print(pre)

        if d >= onthreshold and state == "off":
            state = "on"
            pre = data_x[i]
        if d < onthreshold and state == "on":
            state = "off"
            this_on = data_x[i] - pre
            if this_on < min_on:
                continue
            on.append(this_on.seconds//60)
            if on[-1] == 1:
                print(pre)
            if data_x[i] - pre > timedelta(hours=10):
                print(pre)
    print(min(on))
    print(max(on))
    return sum(on) / (len(on) + 0.000001), on   

def get_freq_cols(df):
    cols = list(df.columns)
    freq = {}
    for c in cols:
        if "Appliance" not in c:
            continue
        freq[c] = df[c].value_counts()
    return freq

In [5]:

filename = "House20.csv"
file_path = os.path.join(root_dir, filename)
meter = pd.read_csv(file_path)
freq_cnt = get_freq_cols(meter)

# y = meter["Appliance7"]
# x = range(len(y))
time = [datetime.strptime(x,datetime_temp) for x in meter["Time"]]
last = 40000
# print(avg_duration_evt(time, y, 0.001))
# plt.ylim(0,0.3)


In [13]:
# print(freq_cnt["Appliance7"])
y = meter["Appliance7"]
avg, on = avg_duration_evt(time, y, 50, min_on = timedelta(minutes=5))
print("The number of on event " + str(len(on)))
print("Average on time " + str(avg))

5
438
The number of on event 1280
Average on time 59.99062495313232


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=y,mode='markers', name='markers'))
fig.show()

In [14]:
from RefitProcessor import RefitProcessor
from config import *
from datetime import timedelta
logging.basicConfig(level=PROJ_LOGGING_LEVEL)

def prepare_refit_data():
    test_project = "refit"
    project_path = os.path.join(DATA_ROOT, test_project)
    device_list = {"House1.csv": [
                        {"name": "Appliance7", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance5", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance8", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House3.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House4.csv": [
                        {"name": "Appliance5", 
                        "onThreshold": 9, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance4", 
                        "onThreshold": 5, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 30, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House8.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 60, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance4", 
                        "onThreshold": 5, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 30, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House9.csv": [
                        {"name": "Appliance3", 
                        "onThreshold": 1, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance5", 
                        "onThreshold": 60, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House15.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 89, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance3", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance6", 
                        "onThreshold": 60, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House18.csv": [
                        {"name": "Appliance7", 
                        "onThreshold": 50, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance5", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance8", 
                        "onThreshold": 50, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
                    "House20.csv": [
                        {"name": "Appliance6", 
                        "onThreshold": 70, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "PC"},
                        {"name": "Appliance4", 
                        "onThreshold": 0, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "WashingMachine"},
                        {"name": "Appliance7", 
                        "onThreshold": 50, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
    }
    refit_processor = RefitProcessor(project_path, {}, device_list)
    ctx_evts, device_evts = refit_processor.preprocess(output_file="p")

prepare_refit_data()

INFO:root:Load REFIT data file: House1.csv
DEBUG:root:The number of device events before filter: {'Appliance7': 2222, 'Appliance5': 4190, 'Appliance8': 7570}
DEBUG:root:The number of device events after filter: {'Appliance7': 1923, 'Appliance5': 927, 'Appliance8': 2886}
INFO:root:Load REFIT data file: House3.csv
DEBUG:root:The number of device events before filter: {'Appliance6': 6000, 'Appliance7': 3707}
DEBUG:root:The number of device events after filter: {'Appliance6': 1506, 'Appliance7': 3241}
INFO:root:Load REFIT data file: House4.csv
DEBUG:root:The number of device events before filter: {'Appliance5': 13593, 'Appliance4': 3078, 'Appliance7': 26191}
DEBUG:root:The number of device events after filter: {'Appliance5': 1045, 'Appliance4': 610, 'Appliance7': 3413}
INFO:root:Load REFIT data file: House8.csv
DEBUG:root:The number of device events before filter: {'Appliance6': 2701, 'Appliance4': 17830, 'Appliance7': 9038}
DEBUG:root:The number of device events after filter: {'Appliance6

In [4]:
import datetime
a = datetime.datetime(2013, 10, 26, 22, 59, 58)
b = datetime.datetime(2013, 10, 27, 0, 0, 1)
